In [1]:
import math
import time
import cv2
from sklearn import neighbors
import numpy as np
import pandas as pd
import os
import os.path
import requests
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
curd = os.getcwd()
try:  
    os.mkdir("{}/known_people".format(curd))
except:
    pass
try:  
    os.mkdir("{}/models".format(curd))
except:
    pass
try:  
    os.mkdir("{}/test".format(curd))
except:
    pass
try:  
    os.mkdir("{}/unknown_pictures".format(curd))
except:
    pass
names=[]

In [2]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [3]:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
   
    X = []
    y = []

    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

### Training data Creation with PhoneCam(Not to be used if IP webcam not installed)

In [ ]:
#Train data creator
import cv2
import numpy as np
import time
import os
curd = os.getcwd()
name = input()


path = "{}/known_people/{}".format(curd,name)
try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
url = "http://192.168.137.184:8080/shot.jpg"
count=0
while count<10:
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content),dtype = np.uint8)
    frame = cv2.imdecode(img_arr,-1)
    cv2.imwrite("{}/{}{}.jpg".format(path,name,count), frame)
    count+=1
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(1)    


    
cv2.destroyAllWindows()

### Generating Training data using WebCam

In [8]:
#Train data creator

name = input()
path = "{}/known_people/{}".format(curd,name)
try:  
    os.mkdir(path)
except OSError:  
    print ("Creation of the directory %s failed" % path)
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 100)
rat, frame = cap.read()

count=0
while count<10:
    
    rat, frame = cap.read()
    cv2.imwrite("{}/{}{}.jpg".format(path,name,count), frame)
    count+=1
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(1)    


cap.release()    
cv2.destroyAllWindows()

RA1711003011322


### Train Model and save it.

In [4]:
#Trainer
if __name__ == "__main__":
    # STEP 1: Train the KNN classifier and save it to disk
    print("Training KNN classifier...")
    classifier = train("known_people",
                       model_save_path="{}/models/trained_knn_model.clf".format(curd),
                       n_neighbors=2)
    print("Training complete!")

   

Training KNN classifier...
Training complete!
